In [31]:
import pandas as pd
data=pd.read_csv('processed_data/data.csv')
import nltk
from nltk.corpus import stopwords
stopwords.words('english')

In [137]:
data=data[data.sender.isin(data.groupby('sender').size().pipe(lambda x: x[x>100]).index)]

In [152]:
splitted=data.body.str.lower().str.replace('[^a-z ]', '').str.split().dropna()

In [153]:
data

,sender,body
64,andy.zipper@enron.com,I would like Amita to have the ability to add ...
65,andy.zipper@enron.com,I think the broker is never really the counter...
66,andy.zipper@enron.com,"Let me try and be clear on this, as I m not su..."
67,andy.zipper@enron.com,FYI
68,andy.zipper@enron.com,Mark can we put togethr a term sheet for the b...
69,andy.zipper@enron.com,FYI
70,andy.zipper@enron.com,Try this one....
71,andy.zipper@enron.com,"This is what I got back from espeed, which is ..."
72,andy.zipper@enron.com,The going gets pretty slow with all the differ...
73,andy.zipper@enron.com,Please take a look at these prior to us gettin...


In [154]:
def b(x):
    try:
        return len(x)
    except:
        print(x)
        return None
splitted.apply(b)

64        43
65        62
66       226
67         1
68        74
69         1
70         3
71        23
72        10
73        11
74        31
75        11
76        19
77        12
78        13
79        74
80         1
81        10
82        11
83        31
84        19
85        12
86        13
87        11
88         4
89        21
90         8
91        56
92        57
93        18
        ... 
86894     19
86895      5
86896     11
86897     12
86898      5
86899     11
86900     10
86901     15
86902      1
86903      5
86904     16
86905     19
86906     75
86907     11
86908     32
86909      7
86910      5
86911     47
86912     12
86913      9
86914     14
86915    286
86916      1
86917     17
86918     11
86919     75
86920      7
86921      3
86922     46
86923     10
Name: body, Length: 86350, dtype: int64

In [155]:
from itertools import chain
from collections import Counter
def get_word_count(x):
    try:
        counted=Counter(chain(*x))
        counter_words=pd.Series(counted)/len(x)
    except:
        print(x)
        raise
    return counter_words

In [156]:
words=get_word_count(splitted).to_frame().rename(columns={0:'baseline'}).query(f'baseline>5/{len(splitted)}')

In [157]:
words_by_sender=data.assign(splitted=splitted).dropna().groupby('sender').splitted.apply(get_word_count).unstack()

In [158]:
rel_count=pd.merge(words_by_sender.T.reset_index(), words.reset_index(), on="index").set_index(
    "index"
).apply(lambda x: x / x.baseline, axis=1)

In [159]:
# Words per email-person, eg how many times does X say W per email, on average
# vs Words per email, how many times an emails says W, on average
%matplotlib inline
rel_count

,andy.zipper@enron.com,barry.tycholiz@enron.com,benjamin.rogers@enron.com,bill.rapp@enron.com,cara.semperger@enron.com,carol.clair@enron.com,charles.weldon@enron.com,chris.dorland@enron.com,chris.germany@enron.com,dan.hyvl@enron.com,...,steven.j.kean@enron.com,susan.bailey@enron.com,susan.scott@enron.com,tana.jones@enron.com,teb.lokey@enron.com,theresa.staab@enron.com,thomas.a.martin@enron.com,tori.kuykendall@enron.com,tracy.geaccone@enron.com,baseline
index,,,,,,,,,,,,,,,,,,,,,
a,0.655751,0.981237,0.779909,0.896421,1.292138,1.198557,1.140530,0.792576,0.798849,1.514127,...,1.085551,0.815398,2.148844,0.962905,0.410672,0.335162,0.656941,1.011896,0.192167,1.0
aa,NaN,1.112872,0.608836,NaN,NaN,1.210210,3.584698,NaN,NaN,NaN,...,0.300876,NaN,2.254788,0.152166,NaN,NaN,NaN,NaN,NaN,1.0
aaa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.451854,NaN,NaN,NaN,NaN,NaN,1.0
aadepartment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,47.839335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
aaron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.739444,NaN,...,3.776790,NaN,NaN,NaN,NaN,NaN,NaN,9.089474,NaN,1.0
ab,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.188291,NaN,NaN,...,1.203505,NaN,NaN,0.152166,NaN,NaN,NaN,NaN,NaN,1.0
aba,NaN,NaN,NaN,NaN,NaN,6.872264,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.912701,NaN,NaN,NaN,NaN,NaN,1.0
abandon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.012700,NaN,NaN,NaN,NaN,NaN,NaN,1.0
abandoned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [194]:
#Not removing stopwords, I or please might be useful
rel_count.isnull().mean(axis=1).pipe(lambda x: x[x<0.01])

index
a         0.0
about     0.0
all       0.0
am        0.0
an        0.0
and       0.0
are       0.0
as        0.0
at        0.0
back      0.0
be        0.0
been      0.0
but       0.0
by        0.0
can       0.0
day       0.0
did       0.0
do        0.0
for       0.0
get       0.0
going     0.0
good      0.0
have      0.0
here      0.0
i         0.0
if        0.0
in        0.0
is        0.0
it        0.0
just      0.0
         ... 
or        0.0
out       0.0
please    0.0
see       0.0
so        0.0
thanks    0.0
that      0.0
the       0.0
there     0.0
they      0.0
think     0.0
this      0.0
time      0.0
to        0.0
today     0.0
until     0.0
up        0.0
us        0.0
want      0.0
was       0.0
we        0.0
what      0.0
when      0.0
who       0.0
will      0.0
with      0.0
work      0.0
would     0.0
you       0.0
your      0.0
Length: 73, dtype: float64

In [191]:
ignore_words=rel_count.isnull().mean(axis=1).pipe(lambda x: x[x>0.8]).index
rel_count[~rel_count.index.isin(ignore_words)].loc['cute'].sort_values()

jeff.dasovich@enron.com         0.382024
james.d.steffes@enron.com       0.508474
carol.clair@enron.com           0.586657
eric.bass@enron.com             0.766969
debra.perlingiere@enron.com     0.859632
kimberly.watson@enron.com       0.918891
baseline                        1.000000
gerald.nemec@enron.com          1.002904
kay.mann@enron.com              1.123851
daren.j.farmer@enron.com        1.207625
matthew.lenhart@enron.com       1.238395
shelley.corman@enron.com        1.464602
john.arnold@enron.com           1.501139
sally.beck@enron.com            2.044755
kevin.hyatt@enron.com           2.064802
marie.heard@enron.com           2.259761
tana.jones@enron.com            2.360434
scott.neal@enron.com            2.404221
jane.tholt@enron.com            2.568412
mark.whitt@enron.com            2.941477
lindy.donoho@enron.com          3.441336
chris.germany@enron.com         3.808495
larry.f.campbell@enron.com      4.026955
stacy.dickson@enron.com         4.387703
stanley.horton@e

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each